<a href="https://colab.research.google.com/github/22222bh/SketchTogether/blob/user%2FByeonghyeon/filtered_bh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

초기 설정

In [1]:
!pip install efficientnet_pytorch
!pip install umap-learn

from google.colab import drive
drive.mount('/content/drive')


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=724ad5f8554da29ca159d0b58987768f7978f1ff1fd4869ede912ac3cabfa76f
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 80 kB 8.5 MB/s 
     |████████████████████████████████| 1.1 MB 30.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76566 sha256=4ce5f76800b52b9bbd7fed0f2aa251009f29afb9cc2a891ea4e122e927b01685
  Stored in directory: /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f
  Created wheel for pynndescent: filename=pynndescent-0.5.4-py3-none-any.whl size=52372 sha256=a2e9a16f79b8ff23b0fbba0760cf5383f391020b184d157738da54a10423c840
  Stored in directory: /root/.cache/pip/wheels/d0/5b/62/3401692ddad12324249c774c4b15ccb046946021e2b581c043
Successfully built umap-learn

In [2]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model
# from keras.applications.vgg16 import preprocess_input 
import os
import logging
import torch
import pickle
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
filename = 'filtered_10'
data_path = '/content/drive/MyDrive/Sketch_RNN/Dataset/Filtered_img/'
cluster_list_path = '/content/drive/MyDrive/Sketch_RNN/Dataset/Filtered_img/cluster_list.txt'
feat_path = "/content/drive/MyDrive/skku_Clustering/features_hw_" + filename + ".npy"
filename_path = "/content/drive/MyDrive/skku_Clustering/filenames_hw_" + filename + ".npy"
save_path = '/content/drive/MyDrive/skku_Clustering/Result/'

In [4]:
os.chdir(data_path)
data_list = []
with os.scandir(data_path) as files:
    for file in files:
        if not file.name.endswith('.txt'):
            data_list.append(file.name)

cluster_list = open(cluster_list_path , 'r').read().split('\n') 

#transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

def feature_extraction(image, model):
    img = tfms(image.convert("RGB")).unsqueeze(0)
    features = model.extract_features(img)
    return features



In [ ]:
from torch import nn
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {}
avg_pooling = nn.AdaptiveAvgPool2d(1)
for folder in data_list:
    image_list = os.listdir(data_path + folder)
    print(image_list)
    print(folder)
    for i, image_name in enumerate(image_list):
      # try:
        image = Image.open(data_path + folder +'/'+ image_name)
        feat = feature_extraction(image, model)
        feat = avg_pooling(feat)
        feat = feat.detach().numpy().reshape(-1)
        name = folder + '_' + str(i)
        data[name] = (feat)

np.save(feat_path, np.array(list(data.values())))
np.save(filename_path, np.array(list(data.keys())))

In [5]:
feat = np.load(feat_path)
#TSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca', random_state=0)
feat = tsne.fit_transform(feat)

dbscan 관련 파라미터는 여기서 수정하면 됩니다.

In [47]:
_eps = 1.7
_ms = 6


def clustering_method(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=cluster_num, n_init = 20, n_jobs=-1, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_

  elif cmd == 'minibatch':
    from sklearn.cluster import MiniBatchKMeans
    mb_kmeans = MiniBatchKMeans(n_clusters=cluster_num,random_state=0, batch_size=6)
    mb_kmeans.fit(input)
    return mb_kmeans.labels_

  elif cmd == 'dbscan':
    from sklearn.cluster import DBSCAN
    dbscan = DBSCAN(eps=_eps, min_samples=_ms) # eps 1.8이면 63프로
    dbscan.fit(input)
    return dbscan.labels_
  
  elif cmd == 'optics':
    from sklearn.cluster import OPTICS, cluster_optics_dbscan
    optics = OPTICS(min_samples = 5)
    optics.fit(input)
    return optics.labels_

labels = clustering_method('dbscan', len(cluster_list), feat)

filenames = np.load(filename_path)
groups = {}
# holds the cluster id and the images { id: [images] }
i = 0
for f, cluster in zip(filenames, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(f)
    else:
        groups[cluster].append(f)


#make cluster_dict for calculate acc
cluster_dict = {}
for cluster in groups:
  image_count = []
  image_name = []
  for image in groups[cluster]:
    image_name.append(image.split('_')[0])
  for name in cluster_list:
    image_count.append(image_name.count(name))
  cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))]
print(len(cluster_dict))
print(cluster_dict)

#Acc
from sklearn.metrics import f1_score
pred = []
gt = []    
for cluster in groups:
    for food in groups[cluster]:
        pred.append(cluster_dict[cluster])
        # gt.append(food.split('_')[3].split('-')[0])
        gt.append(food.split('_')[0])

acc = str(round(f1_score(gt, pred,average='micro') * 100, 2))
print("F1 ACC: " + acc)
save_path = '/content/drive/MyDrive/skku_Clustering/Result_' + str(_eps) + '_' + str(_ms) + '_' + acc + '%_' + str(len(cluster_dict)) + '/'
print(save_path)

27
{1: 'ear', 2: 'ear', 0: 'ear', -1: 'dog', 3: 'ear', 4: 'circle', 6: 'cat', 5: 'dog', 7: 'cat', 8: 'cat', 11: 'dog', 9: 'dog', 10: 'dog', 23: 'eye', 12: 'nose', 19: 'triangle', 13: 'nose', 14: 'triangle', 15: 'moustache', 16: 'moustache', 17: 'cloud', 18: 'triangle', 24: 'mouth', 20: 'triangle', 21: 'cloud', 22: 'eye', 25: 'mouth'}
F1 ACC: 68.2
/content/drive/MyDrive/skku_Clustering/Result_1.7_6_68.2%_27/


In [48]:
# Save img
if not os.path.isdir(save_path):
  print('create dir')
  os.makedirs(save_path)
from PIL import Image        
for cluster in groups:
    for image in groups[cluster]:
        path = save_path + str(cluster) + '_' + str(cluster_dict[cluster]) + '/'
        if os.path.isdir(path) == False:
            os.makedirs(path)
        folder_name = image.split('_')[0] + '/'
        img = Image.open(data_path + folder_name + image + '.jpg')
        img.save(path + image.split('/')[-1] + '.jpg')

create dir
